<a href="https://colab.research.google.com/github/Neetagrg/Mini-Project_2_Spring-2025-WEB-DATA-MINING-CUS-635/blob/main/Mini_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mini Project **2**: Neeta Kumari & Bir Bahadur Gharti Class CUS635

# Install the necessary packages

In [15]:
!pip install --upgrade pip setuptools wheel
!pip install boto3 numpy


# Set Up S3 Access in Google Colab

In [16]:
# Import necessary libraries
import boto3
from botocore.client import Config  # For unsigned access (anonymous requests)

# Team folder for your specific use case
TEAM = "TEAM_6/"  # Set your team folder name here

# Define the provided S3 bucket name
BUCKET_NAME = "cus635-spring2025"  # Replace with your actual bucket name

# Create an anonymous S3 client (No AWS credentials needed for anonymous access)
s3 = boto3.client('s3', config=Config(signature_version='UNSIGNED'))  # No need for AWS access key


In [17]:
# Install the required libraries
!pip install boto3 pandas requests pinecone


# Create a Pinecone index

In [27]:
from pinecone import Pinecone
from pinecone import ServerlessSpec

# Initialize Pinecone with your API key
pc = Pinecone(api_key="pcsk_6akU8Z_2BXXXDSBKbvFCn4sciNM2FeJC6PwAt6wFwQeQjoJKDSjysRbtyBAdUfRv6z87e6")

# Define index name and dimension
index_name = "cus635"
dimension = 1024

# Check if the index already exists
if index_name not in pc.list_indexes().names():
    # Create the index with the specified dimension
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric='euclidean',  # you can choose other metrics like 'cosine', 'dotproduct', etc.
        spec=ServerlessSpec(
            cloud='aws',
            region='us-west-2'  # You can adjust this to your preferred region
        )
    )
    print(f"Index '{index_name}' created successfully!")
else:
    print(f"Index '{index_name}' already exists.")


Index 'cus635' already exists.


# Set up the AWS credentials for Boto3 (Anonymous S3 Access)

In [34]:
import boto3
from botocore.config import Config

# Define your team folder and bucket name
TEAM = "TEAM_6/"
BUCKET_NAME = "cus635-spring2025"

# Create an S3 client with anonymous access
s3 = boto3.client('s3', config=Config(signature_version='UNSIGNED'))

# Try listing objects in the specified folder
response = s3.list_objects_v2(Bucket=BUCKET_NAME, Prefix=TEAM)

# Check and print the contents
if 'Contents' in response:
    print(f"Files in {TEAM} folder:")

    # Loop over the files listed in the S3 bucket
    for obj in response['Contents']:
        file_key = obj['Key']

        # Skip any folders (those that end with '/')
        if file_key.endswith('/'):
            print(f"Skipping folder: {file_key}")
            continue

        print(f"Downloading {file_key}")

        # Download the file from S3
        s3.download_file(BUCKET_NAME, file_key, file_key.split('/')[-1])  # Save it locally with the original file name
        print(f"Downloaded {file_key.split('/')[-1]}")

else:
    print(f"No files found in {TEAM} folder.")


UnknownSignatureVersionError: Unknown Signature Version: UNSIGNED.

In [25]:
pip install pinecone-client transformers torch boto3 numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 139.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nv

# Setup Pinecone Connection

In [29]:
import pinecone

# Initialize Pinecone
pinecone.init(api_key="pcsk_6akU8Z_2BXXXDSBKbvFCn4sciNM2FeJC6PwAt6wFwQeQjoJKDSjysRbtyBAdUfRv6z87e6", environment="gcp-starter")  # Replace with your actual environment

# Connect to your index
index = pinecone.Index("cus635")


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )

